In [36]:
import geopandas as gpd
import pandas as pd

gdf = gpd.read_file("/data/MASTER_dataset CLEANED.geojson")
gdf.geometry.head()

0    POLYGON ((-80.22704 26.16284, -80.22704 26.162...
1    POLYGON ((-80.33135 25.87703, -80.33079 25.877...
2    POLYGON ((-80.33949 25.86636, -80.33935 25.866...
3    POLYGON ((-80.26141 25.35472, -80.26139 25.355...
4    POLYGON ((-80.22994 25.98248, -80.22991 25.982...
Name: geometry, dtype: geometry

In [37]:
gdf_centroids = gdf.copy()
gdf_centroids["geometry"] = gdf_centroids.geometry.centroid

/var/folders/mv/5grglnm12qj9pj0l3s5k20_40000gn/T/ipykernel_59471/1148843992.py:2: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf_centroids["geometry"] = gdf_centroids.geometry.centroid


In [38]:
places = gpd.read_file("/Users/acsoteldo/Desktop/datasets/Data Project 12 Dec 2025/data/flcities/tl_2025_12_place.shp")

In [39]:
places = places[[
    "NAME",        # City name
    "NAMELSAD",    # Full label
    "geometry"
]]

In [40]:
places = places.to_crs(gdf_centroids.crs)

In [41]:
gdf_city = gpd.sjoin(
    gdf_centroids,
    places,
    how="left",
    predicate="within"
)

In [42]:
gdf_city = gdf_city.rename(columns={
    "NAME": "city_name",
    "NAMELSAD": "city_label"
})

In [43]:
gdf_city = gdf_city.drop(columns=["index_right"])

In [44]:
zcta = gpd.read_file("/Users/acsoteldo/Desktop/datasets/Data Project 12 Dec 2025/data/zipcodes/tl_2025_us_zcta520.shp")

In [45]:
zcta = zcta[[
    "ZCTA5CE20",  # ZIP code
    "geometry"
]]

In [46]:
zcta = zcta.to_crs(gdf_city.crs)

In [47]:
gdf_city_zip = gpd.sjoin(
    gdf_city,
    zcta,
    how="left",
    predicate="within"
)

In [48]:
gdf_city_zip = gdf_city_zip.rename(columns={
    "ZCTA5CE20": "zcta"
})

In [49]:
gdf_city_zip = gdf_city_zip.drop(columns=["index_right"])

In [50]:
gdf_final = gdf_city_zip.drop(columns="geometry").merge(
    gdf[["geoid", "geometry"]],
    on="geoid",
    how="left"
)

gdf_final = gpd.GeoDataFrame(gdf_final, geometry="geometry", crs=gdf.crs)

In [51]:
gdf_final[["city_name", "zcta"]].isna().mean()

city_name    0.165138
zcta         0.010485
dtype: float64

In [52]:
gdf_final["city_name"].value_counts().head(10)

city_name
Miami              129
Hialeah             57
Fort Lauderdale     46
Hollywood           36
West Palm Beach     31
Pembroke Pines      31
Boca Raton          28
Miami Gardens       28
Pompano Beach       27
Miramar             24
Name: count, dtype: int64

In [53]:
gdf_final["zcta"].value_counts().head(10)

zcta
33186    19
33012    19
33015    18
33311    17
33157    17
33411    15
33178    15
33176    15
33033    15
33125    15
Name: count, dtype: int64

In [54]:
gdf_final.to_file(
    "/data/MASTER_dataset CLEANED v2.geojson",
    driver="GeoJSON"
)

In [56]:
gdf_final.drop(columns="geometry").to_csv("/data/MASTER_dataset CLEANED v2.csv", index=False)